In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd


# Customize matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'font.size': 22,
        'ytick.color' : 'black',
        "xtick.color" : 'black',
        'axes.labelcolor' : 'black',
        'axes.edgecolor' : 'black'
	}
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
n_points = 50
n_samples = 1
vec_alpha = np.linspace(0, 1, n_points*n_samples)

EL = -60*mV
EE = 0*mV
EI = -80*mV
tauA = 1*ms
tauN = 100*ms
tauI = 10*ms
tauL = 20*ms

eqs = '''
dv/dt = ((EL - v) + ((1 - alpha)*gA + alpha*gN)*(EE - v) + gI*(EI - v))/tauL : volt
dgA/dt = -gA/tauA : 1
dgN/dt = -gN/tauN : 1
dgI/dt = -gI/tauI : 1
alpha : 1
'''

rate_list = [5, 10, 20]
mean_V = {'alpha':vec_alpha, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples)}
var_V = {'alpha':vec_alpha, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples)}

for rate in rate_list:
	print(rate, end='\t')
	for i, alpha_value in enumerate(vec_alpha):
		print(i/size(vec_alpha), end=' ')
		
		start_scope()
		defaultclock.dt = 0.01*ms
		
		G = NeuronGroup(1, eqs, method='rk4')
		GPA = PoissonGroup(400, rates=rate*Hz)
		GPN = PoissonGroup(400, rates=rate*Hz)
		GPI = PoissonGroup(100, rates=rate*Hz)

		SA = Synapses(GPA, G, 'w : 1', on_pre='gA += w')
		SN = Synapses(GPN, G, 'w : 1', on_pre='gN += w')
		SI = Synapses(GPI, G, 'w : 1', on_pre='gI += w')
		SA.connect()
		SN.connect()
		SI.connect()

		G.v = EL
		SA.w = 0.1
		SN.w = 0.1
		SI.w = 0.4

		G.alpha = alpha_value
		
		run(5000*ms)

		M = StateMonitor(G, 'v', record=True)

		run(20000*ms)

		mean_V[rate][i] = np.mean(M.v[0]/mV)
		var_V[rate][i] = np.var(M.v[0]/mV)
		
	print()

5	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 
10	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 
20	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 


In [3]:
data_directory_path = "../../Data/V_data/"

# Convert to pandas DataFrame
df_mean_V = pd.DataFrame.from_dict(mean_V)
df_var_V = pd.DataFrame.from_dict(var_V)


# Save Dataframe do csv file
df_mean_V.to_csv(f"{data_directory_path}mean_V_interpolated.dat")
df_var_V.to_csv(f"{data_directory_path}var_V_interpolated.dat")


In [6]:
var_V

{'tauA': array([5.03368169e-01, 4.69711562e-01, 9.08143886e-01, 8.17433972e-01,
        9.94229950e-01, 4.46901273e-01, 7.29482595e-01, 4.91118235e-01,
        4.59225207e-01, 3.30970752e-01, 3.31848470e-01, 6.82099298e-01,
        6.09945049e-01, 6.90896120e-01, 8.56156687e-01, 3.92172859e-01,
        3.27960489e-01, 7.56096630e-01, 6.65253298e-01, 7.03513232e-01,
        3.69585920e-01, 4.01030555e-01, 9.84646778e-01, 9.38977241e-02,
        7.74140092e-01, 4.60130031e-01, 9.29382164e-01, 3.72816119e-01,
        6.29748051e-02, 1.44457089e-02, 7.14055401e-02, 8.99193907e-01,
        5.98253332e-02, 6.95869086e-01, 8.20324791e-01, 1.63160457e-01,
        3.07749619e-01, 1.34749981e-01, 8.13229827e-01, 1.64300185e-01,
        9.28874281e-01, 4.79588086e-01, 8.19503978e-01, 2.12183440e-01,
        1.43177187e-01, 8.94829950e-01, 5.83859526e-01, 1.89527498e-01,
        2.71031267e-01, 3.71593409e-01, 8.15960275e-01, 7.84663294e-01,
        4.79949909e-02, 1.91179958e-01, 2.09820907e-01, 